# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (linear problem), with wrong boundary markers

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **weak imposition of Dirichlet BCs**: this requires an introduction of a multiplier $\lambda$ which is restricted to $\Gamma$, and solves
$$
\text{find } w, \lambda \in V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla w \cdot \nabla v + \int_\Gamma \lambda v = \int_\Omega f v, & \forall v \in V,\\
\int_\Gamma w \mu = \int_\Gamma g \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

This example (accompanied by the corresponding DOLFIN 2020 one at the other link) assesses the robustness of the variable restriction to errors in the marking procedure.

In [ ]:
import matplotlib
import matplotlib.collections
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpi4py import MPI
from petsc4py import PETSc
from ufl import dx, grad, inner, Measure, TestFunction, TrialFunction
from dolfinx import DirichletBC, Function, FunctionSpace, MeshTags, solve, UnitSquareMesh
from dolfinx.cpp.mesh import midpoints
from dolfinx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                         BlockVecSubVectorWrapper, create_vector_block, DofMapRestriction,
                         locate_dofs_topological)
from dolfinx.plotting import plot

### Tolerance for boundary marking

In [ ]:
tol = np.finfo(float).eps
# tol = 0.4

### Auxiliary functions for plotting (copied from tutorial 07)

In [ ]:
def plot_mesh(mesh, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_aspect("equal")
    points = mesh.geometry.x
    cells = mesh.geometry.dofmap.array().reshape((-1, mesh.topology.dim + 1))
    tria = tri.Triangulation(points[:, 0], points[:, 1], cells)
    ax.triplot(tria, color="k")
    return ax

In [ ]:
def plot_mesh_tags(mesh_tags, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_aspect("equal")
    mesh = mesh_tags.mesh
    points = mesh.geometry.x
    colors = ["b", "r"]
    cmap = matplotlib.colors.ListedColormap(colors)
    cmap_bounds = [0, 0.5, 1]
    norm = matplotlib.colors.BoundaryNorm(cmap_bounds, cmap.N)
    assert mesh_tags.dim in (mesh.topology.dim, mesh.topology.dim - 1)
    if mesh_tags.dim == mesh.topology.dim:
        cells = mesh.geometry.dofmap.array().reshape((-1, mesh.topology.dim + 1))
        tria = tri.Triangulation(points[:, 0], points[:, 1], cells)
        cell_colors = np.zeros((cells.shape[0], ))
        cell_colors[mesh_tags.indices[mesh_tags.values == 1]] = 1
        mappable = ax.tripcolor(tria, cell_colors, edgecolor="k", cmap=cmap, norm=norm)
    elif mesh_tags.dim == mesh.topology.dim - 1:
        tdim = mesh.topology.dim
        geometry_dofmap = mesh.geometry.dofmap
        cells_map = mesh.topology.index_map(mesh.topology.dim)
        num_cells = cells_map.size_local + cells_map.num_ghosts
        connectivity_cells_to_facets = mesh.topology.connectivity(tdim, tdim - 1)
        connectivity_cells_to_vertices = mesh.topology.connectivity(tdim, 0)
        connectivity_facets_to_vertices = mesh.topology.connectivity(tdim - 1, 0)
        vertex_map = {topology_index: geometry_index
                      for c in range(num_cells)
                      for (topology_index, geometry_index) in zip(
                          connectivity_cells_to_vertices.links(c), geometry_dofmap.links(c))}
        linestyles = ["solid", "solid"]
        lines = list()
        lines_colors_as_int = list()
        lines_colors_as_str = list()
        lines_linestyles = list()
        mesh_tags_1 = mesh_tags.indices[mesh_tags.values == 1]
        for c in range(num_cells):
            facets = connectivity_cells_to_facets.links(c)
            for f in facets:
                if f in mesh_tags_1:
                    value_f = 1
                else:
                    value_f = 0
                vertices = [vertex_map[v] for v in connectivity_facets_to_vertices.links(f)]
                lines.append(points[vertices][:, :2])
                lines_colors_as_int.append(value_f)
                lines_colors_as_str.append(colors[value_f])
                lines_linestyles.append(linestyles[value_f])
        mappable = matplotlib.collections.LineCollection(lines, cmap=cmap, norm=norm,
                                                         colors=lines_colors_as_str,
                                                         linestyles=lines_linestyles)
        mappable.set_array(np.array(lines_colors_as_int))
        ax.add_collection(mappable)
        ax.autoscale()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(mappable, cax=cax, cmap=cmap, norm=norm, boundaries=cmap_bounds, ticks=cmap_bounds)
    return ax

### Helper function for (possibly wrong!) boundary marking

In [ ]:
def mark_boundary(tol):
    tdim = mesh.topology.dim
    cells_map = mesh.topology.index_map(mesh.topology.dim)
    num_cells = cells_map.size_local + cells_map.num_ghosts
    mesh.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    facets_map = mesh.topology.index_map(mesh.topology.dim - 1)
    num_facets = facets_map.size_local + facets_map.num_ghosts
    connectivity_cells_to_facets = mesh.topology.connectivity(tdim, tdim - 1)
    facet_midpoints = midpoints(mesh, tdim - 1, range(num_facets))

    def on_boundary(x):
        def near(x, a, tol):
            return np.abs(x - a) < tol

        return np.logical_or(
            np.logical_or(near(x[0], 0.0, tol), near(x[0], 1.0, tol)),
            np.logical_or(near(x[1], 0.0, tol), near(x[1], 1.0, tol)))

    boundary_facets = list()
    for c in range(num_cells):
        facets = connectivity_cells_to_facets.links(c)
        for f in facets:
            if on_boundary(facet_midpoints[f]):
                boundary_facets.append(f)
    boundary_facets = np.array(boundary_facets)
    boundary_values = np.ones(boundary_facets.shape, dtype=np.int32)

    return MeshTags(mesh, mesh.topology.dim - 1, boundary_facets, boundary_values)

### Mesh

In [ ]:
mesh = UnitSquareMesh(MPI.COMM_WORLD, 32, 32)
boundaries = mark_boundary(tol)
facets_Gamma = boundaries.indices[boundaries.values == 1]

In [ ]:
plot_mesh(mesh)

In [ ]:
plot_mesh_tags(boundaries)

In [ ]:
# Define associated measures
ds = Measure("ds")(subdomain_data=boundaries)

### Weak (and possibly wrong!) imposition of Dirichlet BCs

In [ ]:
# Define a function space
V = FunctionSpace(mesh, ("Lagrange", 2))
M = V.clone()

In [ ]:
# Define restrictions
dofs_V = np.arange(0, V.dofmap.index_map.block_size * (
    V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts))
dofs_M_Gamma = locate_dofs_topological(M, boundaries.dim, facets_Gamma)
restriction_V = DofMapRestriction(V.dofmap, dofs_V)
restriction_M_Gamma = DofMapRestriction(M.dofmap, dofs_M_Gamma)
restriction = [restriction_V, restriction_M_Gamma]

In [ ]:
# Define trial and test functions
(u, l) = (TrialFunction(V), TrialFunction(M))
(v, m) = (TestFunction(V), TestFunction(M))

In [ ]:
# Define problem block forms
g = Function(V)
g.interpolate(lambda x: np.sin(3 * np.pi * x[0] + 1) * np.sin(3 * np.pi * x[1] + 1))
a = [[inner(grad(u), grad(v)) * dx, l * v * ds],
     [u * m * ds, None]]
f = [v * dx, g * m * ds]

In [ ]:
# Assemble the block linear system
A = assemble_matrix_block(a, bcs=[], restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=[], restriction=restriction)

In [ ]:
# Solve
ul = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, ul)
ul.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
print(ul.norm())

In [ ]:
# Split the block solution in components
(u, l) = (Function(V), Function(M))
with BlockVecSubVectorWrapper(ul, [V.dofmap, M.dofmap], restriction) as ul_wrapper:
    for ul_wrapper_local, component in zip(ul_wrapper, (u, l)):
        with component.vector.localForm() as component_local:
            component_local[:] = ul_wrapper_local

In [ ]:
plot(u)

In [ ]:
plot(l)

### Strong (and correct) imposition of Dirichlet BCs for comparison

In [ ]:
# Define Dirichlet BC object on Gamma
boundaries_ex = mark_boundary(np.finfo(float).eps)
facets_Gamma_ex = boundaries_ex.indices[boundaries_ex.values == 1]
dofs_V_Gamma_ex = locate_dofs_topological(V, boundaries.dim, facets_Gamma_ex)
bc_ex = DirichletBC(g, dofs_V_Gamma_ex)

In [ ]:
# Solve
u_ex = Function(V)
solve(a[0][0] == f[0], u_ex, bc_ex,
      petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_ex), grad(u_ex)) * dx), op=MPI.SUM))
err_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_ex - u), grad(u_ex - u)) * dx), op=MPI.SUM))
print("Relative error is equal to", err_norm / u_ex_norm)